## Main Workspace

Connect to the main workspace that demo as the main workspace of a professional ML engineer.

**CLI**

`az account set -s "ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9"`

`az configure --defaults group=ZiqiTest workspace=ZiqiPipelineTest location=eastus`

In [4]:
#import required libraries for workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#import required libraries for environments examples
from azure.ai.ml.entities import Environment, BuildContext

#Enter details of your AzureML workspace
subscription_id = 'ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9'
resource_group = 'ZiqiTest'
workspace = 'ZiqiPipelineTest'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

## Create Component

We have provided a sample component in the `component` folder. There is a `train.yml` file defining 
the component and a `train_scr/train.py` file defining the component logic. 

## Create Registry

Registries enable sharing of assets like components, environments, and models across multiple teams and workspaces in your organization. See details at https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-registries?tabs=portal

There are three ways to create a registry:
* From Azure CLI.
* From Azure Portal.
* From AzureML Studio.

If you are using Azure CLI, you should define the registry with a YAML file first and you can create a registry using the following command:
`az ml registry create --file registry.yml`

**Make sure you've updated to the latest CLI version to run this CLI command.** To check yourCLI version, run `az --version`. If you are using an older version, please update to the latest version by running `az upgrade`.

## Register Component

Components are reusable building blocks of Machine Learning pipelines in AzureML. Creating a component in a registry allows you to use the component in any pipeline in any workspace within your organization. See details at https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=cli#create-a-component-in-registry

There are two ways to register a component:
* From Azure CLI.
* From Python SDK.

We have provided a sample component in the `component` folder. There is a `train.yml` file defining 
the component and a `train_scr/train.py` file defining the component logic. 
You can register the component using the following command:
`az ml component create --file train.yml --registry-name <registry-name>`

In [9]:
from azure.ai.ml import load_component
train_model = load_component(source="Components/train.yml")
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '2'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
code: ./train_src
is_deterministic: true
tags: {}



In [10]:
ml_client_registry = MLClient (credential=credential,
        registry_name = "ZiqiTestRegistry")
ml_client_registry.components.create_or_update(train_model)

Uploading train_src (0.0 MBs): 100%|##########| 2614/2614 [00:00<00:00, 10187.60it/s]


Creating/updating registry component train_linear_regression_model with version 2 

.

Done (0m 6s)


CommandComponent({'auto_increment_version': False, 'source': 'REMOTE.REGISTRY', 'is_anonymous': False, 'name': 'train_linear_regression_model', 'description': None, 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ZiqiTestRegistry/components/train_linear_regression_model/versions/2', 'Resource__source_path': None, 'base_path': WindowsPath('.'), 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x000002023B263B80>, 'serialize': <msrest.serialization.Serializer object at 0x000002023B2D4DF0>, 'command': 'python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}', 'code': 'azureml://registries/ZiqiTestRegistry/codes/9b0e58fc-67e8-4416-9749-4341e471f336/versions/1', 'environment_variables': None, 'environment': 'azureml://registries/azureml/environments/AzureML-sklearn-0.24-ubuntu18.04-py37-cpu

## Switch to a different workspace

If you want to switch to a different workspace, use
`az configure --default  workspace=ZiqiPipelineTest-Client`

In [11]:
#import required libraries for workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#import required libraries for environments examples
from azure.ai.ml.entities import Environment, BuildContext

#Enter details of your AzureML workspace
subscription_id = 'ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9'
resource_group = 'ZiqiTest'
workspace = 'ZiqiPipelineTest-Client'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

## Run a pipeline job in a different workspace using a component from a registry

You can run a pipeline job in a different workspace using a component from a registry. See details at https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=cli#run-a-pipeline-job-in-a-workspace-using-a-component-from-a-registry


Run the pipeline job with the az ml job create command.
`az ml job create --file single-job-pipeline.yml`

In this pipeline, component is retrived from
`component: azureml://registries/ZiqiTestRegistry/components/train_linear_regression_model/versions/1`

In [13]:
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourceGroups/ZiqiTest/providers/Microsoft.MachineLearningServices/workspaces/ZiqiPipelineTest-Client/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'c:\\Users\\ziqiwang\\OneDrive - Microsoft\\Documents\\GitHub\\azureml-examples\\Ziqi-Test', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002023B252C40>, 'resource_id': None, 'location': 'eastus2', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 10, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x000002023B257280>, 'tier': 'dedicated', 'subnet': None})


In [17]:
ml_client_registry = MLClient (credential=credential,
        registry_name = "ZiqiTestRegistry")

<iterator object azure.core.paging.ItemPaged at 0x20238a7bcd0>

In [23]:
for i in ml_client_registry.components.list():
    print(i)

Component({'auto_increment_version': True, 'source': 'REMOTE.REGISTRY', 'is_anonymous': False, 'name': 'train_linear_regression_model', 'description': '', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ZiqiTestRegistry/components/train_linear_regression_model', 'Resource__source_path': None, 'base_path': None, 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000020238AAB190>, 'serialize': <msrest.serialization.Serializer object at 0x000002023A4E5610>, 'version': None, 'latest_version': None, 'schema': None, 'type': '_container', 'display_name': None, 'is_deterministic': None, 'inputs': {}, 'outputs': {}, 'yaml_str': None, 'other_parameter': {}})
Component({'auto_increment_version': True, 'source': 'REMOTE.REGISTRY', 'is_anonymous': False, 'name': 'train', 'description': '', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ZiqiTestRegistry/components/train', 'Resource__source_path': None, 'base_path': None, 'creation_context': <azure.ai.

In [25]:
train_component_from_registry = ml_client_registry.components.get(name='train_linear_regression_model')

In [26]:
# Construct a pipeline with two steps.
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input
parent_dir = ""

@pipeline()
def pipeline_with_registered_components(training_data):
    train_job = train_component_from_registry(
        training_data=training_data,
    )
pipeline_job = pipeline_with_registered_components(
    training_data=Input(type="uri_folder", path=parent_dir + "/data_transformed/"),
)
pipeline_job.settings.default_compute = "cpu-cluster"
print(pipeline_job)

display_name: pipeline_with_registered_components
type: pipeline
inputs:
  training_data:
    type: uri_folder
    path: azureml:/data_transformed/
outputs: {}
jobs:
  train_job:
    type: command
    inputs:
      training_data:
        path: ${{parent.inputs.training_data}}
    outputs: {}
    code: azureml://registries/ZiqiTestRegistry/codes/9b0e58fc-67e8-4416-9749-4341e471f336/versions/1
    resources:
      instance_count: 1
      properties: {}
    environment_variables: {}
    properties: {}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: train_linear_regression_model
      version: '2'
      display_name: TrainLinearRegressionModel
      type: command
      inputs:
        training_data:
          type: uri_folder
        test_split_ratio:
          type: number
          default: '0.2'
          max: '1.0'
          min: '0.0'
      outputs:
        model_output:
          type: mlflow_model
        test_data:


In [29]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="sdk_job_component_from_registry"
)
pipeline_job

Exception: [31m

Error: 

1) One or more files or folders do not exist.


Details: 

There is no dir on target path: /data_transformed/

Resolutions:
Double-check the directory paths you provided and enter the correct paths.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command
Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: https://code.visualstudio.com/docs/datascience/azure-machine-learning. To set up VS Code, visit https://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code
[39m